In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)

def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

image_data = {
    "A": [
        [0,1,0],
        [1,0,1],
        [1,1,1]
    ],
    "L": [
        [1,0,0],
        [1,0,0],
        [1,1,1]
    ],
    "0": [
        [1,1,1],
        [1,0,1],
        [1,1,1]
    ],
    "1": [
        [0,1,0],
        [1,1,0],
        [0,1,0]
    ],
    "T": [
        [1,1,1],
        [0,1,0],
        [0,1,0]
    ]
}

def add_noise(image, count=10):
    variants = []
    for _ in range(count):
        variant = np.array(image) + np.random.randint(0, 2, (3, 3))
        variant = np.clip(variant, 0, 1)
        variants.append(variant.flatten())
    return variants

classes = list(image_data.keys())
X = []
y = []
for label_index, label in enumerate(classes):
    base_image = image_data[label]
    variants = add_noise(base_image, count=10)
    X.extend(variants)
    y.extend([one_hot(label_index, len(classes)) for _ in variants])

X = np.array(X)
y = np.array(y)

Neural Network

In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output, learning_rate):
        output_error = output - y
        output_delta = output_error * sigmoid_derivative(output)

        hidden_error = np.dot(output_delta, self.W2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)

        self.W2 -= np.dot(self.a1.T, output_delta) * learning_rate
        self.b2 -= np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.W1 -= np.dot(X.T, hidden_delta) * learning_rate
        self.b1 -= np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs=1000, learning_rate=0.1):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

nn = NeuralNetwork(input_size=9, hidden_size=10, output_size=5)
nn.train(X, y, epochs=1000, learning_rate=0.1)

Epoch 0, Loss: 0.3155
Epoch 100, Loss: 0.0820
Epoch 200, Loss: 0.0660
Epoch 300, Loss: 0.0593
Epoch 400, Loss: 0.0562
Epoch 500, Loss: 0.0544
Epoch 600, Loss: 0.0531
Epoch 700, Loss: 0.0518
Epoch 800, Loss: 0.0507
Epoch 900, Loss: 0.0499


In [3]:
print("\nPredictions on sample inputs:")
for i, label in enumerate(classes):
    sample = np.array(image_data[label]).flatten().reshape(1, -1)
    prediction = nn.predict(sample)
    print(f"Expected: {label}, Predicted: {classes[prediction[0]]}")


Predictions on sample inputs:
Expected: A, Predicted: A
Expected: L, Predicted: L
Expected: 0, Predicted: 0
Expected: 1, Predicted: 1
Expected: T, Predicted: T
